# Exercice 2 : Diffusion de particules

Dans cet exercice, vous allez simuler la diffusion de particules dans un milieu plan, comme dans le cas présenté dans la dernière vidéo.

Les grandes lignes de l'algorithme sont les suivantes :

Afin d'alléger la programmation, nous avons choisi, lorsqu'une particule à déplacer appartient à la source d'ajouter sa destination à la liste des particules plutôt que la déplacer puis la remplacer. Ces deux méthodes reviennent exactement au même.

## 1- Initialisation

Commencez par importer les librairies nécessaires.

In [8]:
# Importation des librairies
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import animation, rc
import matplotlib.colors as colors
import random

Il vous faut maintenant définir le milieu dans lequel les particules vont évoluer. Il se caractérise simplement par ses dimensions.

In [9]:
# Définition des dimensions du milieu
h = 5 # hauteur du milieu
L = 8 # largeur du milieu

Choisissez également combien de temps durera la simulation (en nombre de pas de temps).

In [10]:
T = 10 # durée de la simulation en nombre de pas de temps

Définissons la source des particules. C'est la seule zone du milieu qui contient des particules au démarrage et toutes les particules quittant cette zone sont immédiatement remplacées.

Commencez par traiter le cas présenté dans la dernière vidéo : la  source est la ligne du bas du milieu.

Nous stockerons les points de la sources sous la forme d'un tableau contenant des tableaux de taille deux. Ces tableaux représenteront les coordonnées des points dans le milieux, la première coordonnée étant la hauteur à laquelle se situe la particule en partant du haut et la deuxième coordonnée étant la largeur à laquelle elle se situe en partant de la gauche.

Créez votre source de particules.

In [11]:
source = [[h-1, i] for i in range(L)]

Lors des pas de temps successifs les particules vont se déplacer, il nous faudra donc stoker les différentes coordonnées des particules pour tout instant. Vous les stockerez exactement sous la même forme que les particules de source. 

À l'instant t = 0 les seules particules présentes sont celles de la source.

In [12]:
ensemble_particules = source.copy()

Il faut finalement créer le tableau qui stockera toutes les particules existantes pour chaque instant.

In [13]:
stockage_ensemble_particules = [[-1]] * T

## 2- Simulation

Maintenant que nos variables sont initialisées, il nous faut simuler la diffusion. Cette partie n'implique pas l'affichage des résultats obtenus.

### 2-1- Choix des particules à déplacer

Il vous faut définir la fonction qui choisira les particules à déplacer. Ces particules doivent être choisies aléatoirement parmis la liste des particules présentes dans le milieu. Vous utiliserez pour cela la fonction sample de la bibliothèque random. 

Commencez par utiliser la fonction help pour obtenir les spécificaion de random.sample.

In [14]:
help(random.sample)

Help on method sample in module random:

sample(population, k) method of random.Random instance
    Chooses k unique random elements from a population sequence or set.
    
    Returns a new list containing elements from the population while
    leaving the original population unchanged.  The resulting list is
    in selection order so that all sub-slices will also be valid random
    samples.  This allows raffle winners (the sample) to be partitioned
    into grand prize and second place winners (the subslices).
    
    Members of the population need not be hashable or unique.  If the
    population contains repeats, then each occurrence is a possible
    selection in the sample.
    
    To choose a sample in a range of integers, use range as an argument.
    This is especially fast and space efficient for sampling from a
    large population:   sample(range(10000000), 60)



Il vous faudra choisir une certaine quantité de particules à sélectionner. Pour cela commencez par définir la proportion des particules qui doit être sélectionnée.

In [15]:
# proportion désigne la proportion des particules qui doivent être 
# sélectionnées pour être déplacées à chaque pas de temps.
# proportion est donc un flottant compris entre 0 et 1.
proportion = 0.5 

Créons maintenant la fonction choisis_particules.

In [16]:
def choisis_particules(ensemble_particules, proportion) :
    """
    Prérequis :
    ensemble_particules est un tableau de particules, une particule étant un 
    tableau d'entiers de taille 2 dont les deux éléments sont des entiers 
    strictement inférieurs à N.
    proportion est un flottant compris entre 0 et 1.
    
    Renvoie une liste de particules choisies aléatoirement dans ensemble_particules
    de longueur partie entière de proportion * ensemble_particules
    """
    # On commence par des tests sur la valeur de proportion.
    assert(proportion >= 0)
    assert(proportion <= 1)
    
    # Remplissez la fonction
    k = int(len(ensemble_particules) * proportion)
    return random.sample(ensemble_particules, k)
    

Testez votre fonction sur l'ensemble de particules initial pour différentes valeurs de proportion. Testez notamment les cas limites.

In [17]:
# Testez choisis_particules
h = 3
L = 4
ensemble_particules = [[0, 1], [1, 2], [2, 1], [2, 2], [2, 3]]
print(choisis_particules(ensemble_particules, 0))
print(choisis_particules(ensemble_particules, 0.5))
print(choisis_particules(ensemble_particules, 1))
print(choisis_particules(ensemble_particules, 0.33333))

[]
[[2, 2], [0, 1]]
[[2, 1], [2, 2], [2, 3], [1, 2], [0, 1]]
[[2, 3]]


### 2-2- Déterminer la destination des particules

Si une particule a été sélectionnée pour se déplacer, il faut choisir quelle sera sa destination. Pour rappel une particule doit aller aléatoirement dans une de ses destinations possibles, ses destinations possibles étant les cases du dessus, du dessous et des côté qui ne sont pas occupées et font partie du milieu. LES CASES EN DIAGONALES N'EN FONT PAS PARTIE.
Si aucune destination possible n'existe, une copie de particule est renvoyée.

Remplissez la fonction trouve_destination.

In [270]:
def trouve_destination(particule, ensemble_particules, h, L) :
    """
    Prérequis :
    particule est un tableau d'entiers positif à deux éléments. particule doit
    appartenir au milieu.
    ensemble_particules est un tableau de tableaux d'entiers à deux éléments 
    appartenant au milieu. ensemble_particules contient particule.
    h et L sont des entiers strictement positifs définissant la taille du milieu
    
    Renvoie un tableau à deux éléments représentant la destination de particule.
    Cette destination est aléatoirement choisie parmis les destinations possibles 
    qui sont les cases du dessus, du dessous et des côté de celles de particules
    si elles existent et ne sont pas occupées. 
    Si aucune destination possible n'existe, une copie de particule est renvoyée.
    """
    # Test d'une des préconditions
    assert(particule in ensemble_particules)
    
    ##############################################################################
    #                                 CONSEILS                                   #
    # Sélectionnez d'abord les destinations possibles et créez un tableau les    #
    # contenant. Veillez à bien vérifier que ces destinations ne soient pas      #
    # occupées et à ce qu'elles fassent bien partie du milieu. Vous n'avez       #
    # besoin pour cela que des coordonnées de particule.                         #
    # Vous pourez ensuite aléatoirement choisir un élément parmis les            #
    # destinations possibles. Veillez à bien gérer le cas où il n'y en a pas.    #
    ##############################################################################
    
    
    # Détermination des destinations possibles - À remplir
    [l, c] = particule
    
    voisins_directs = [[l + 1, c], [l - 1, c], [l, c + 1], [l, c - 1]]
    destinations_possibles = []
    for v in voisins_directs :
        lv, cv = v
        if (lv >= 0) and (lv < h) and (cv >= 0) and (cv < L) and (not v in ensemble_particules) :
            destinations_possibles.append(v.copy())
    
    # Choix et renvoi de la destination - À remplir
    if len(destinations_possibles) > 0 :
        ret = random.sample(destinations_possibles, 1)[0]

        return random.sample(destinations_possibles, 1)[0]
    else :
        return particule



Testez votre fonction à l'aide des cas suivants. Pour vous assurer que le résultat est correct, faites un dessin du milieu.
TESTEZ TOUS LES CAS LIMITES PRÉSENTÉS DANS LES PRÉCONDITIONS.

Une erreur à ce stade du code peut s'avérer très problématique pour la suite et difficile à détecter. Il est donc important d'effectuer correctement ce type de tests.

In [271]:
h = 7 
L = 3
ensemble_particules = [[0, 0], [1, 1], [2, 0], [2, 1], [2, 2], [3, 1], [3, 2], [5, 1]]
print(trouve_destination([0, 0], ensemble_particules, h, L)) # dans un coin 
print(trouve_destination([2, 1], ensemble_particules, h, L)) # coincée entre quatre particules
print(trouve_destination([2, 2], ensemble_particules, h, L)) # une seule destination possible
print(trouve_destination([5, 1], ensemble_particules, h, L)) # quatre destinations possibles

[1, 0]
[2, 1]
[1, 2]
[4, 1]


### 2-3- Déplacer une particule

Plaçons nous désormais dans le cas où nous devons déplacer une particule et en connaissons la destination. Il faut alors modifier le tableau ensemble particule de la manière suivante :
    - Si la particule à déplacer appartient à la source, on ajoute la destination à la liste des particules
    - Sinon on modifie la particule en veillant à ne pas causer de problème d'aliasing (comme la modification des données précédemment enregistrées). 
    
Remplissez la fonction deplace_particule.

In [272]:
def deplace_particule(particule, destination) :
    """
    Prérequis : 
    particule et destination sont des particules, c'est à dire des tableaux d'entiers
    à deux éléments. destination n'appartient pas à ensemble_particules, définie dans 
    le code.
    
    Si particule et destination sont les même, on ne fait rien.
    Sinon :
        Si particule appartient à source :
            ajoute destination à ensemble_particules
        Sinon :
            remplace l'élément particule de ensemble_particules par destination.
        
    Ne renvoie rien.
    """
    #################################################################################
    #                                    CONSEILS                                   #
    # On pourra utiliser la méthode index qui s'applique aux tableau.               #
    # tab.index(element) est l'index de element dans le tableau tab. La fonction    #
    # lève une exception si element n'appartient pas à tab. Pour plus de précision  #
    # utilisez la commande suivante :                                               #
    # help(list.index)                                                              #
    #################################################################################
    
        
    # Remplissez la suite
    if particule != destination :
        if particule in source :
            assert(not destination in ensemble_particules)
            ensemble_particules.append(destination)
        else :
            indice = ensemble_particules.index(particule)
            ensemble_particules[indice] = destination

Testez votre fonction dans différents cas de figure.

In [273]:
h = 7 
L = 3
source = [[h-1, i] for i in range(L)]
ensemble_particules = [[0, 0], [1, 1], [2, 0], [2, 1], [2, 2], [3, 1], [3, 2], [6, 0], [6, 1], [6, 2]]


# Test pour une particule hors de la source
particule = [1, 1]
destination = [0, 1]
deplace_particule(particule, destination)
print(ensemble_particules)

# Test pour une particule dans la source
particule = [6, 0]
destination = [5, 0]
deplace_particule(particule, destination)
print(ensemble_particules)

# D'autres idées ?
particule = [0, 0]
destination = [0, 1]
deplace_particule(particule, destination)
print(ensemble_particules)

[[0, 0], [0, 1], [2, 0], [2, 1], [2, 2], [3, 1], [3, 2], [6, 0], [6, 1], [6, 2]]
[[0, 0], [0, 1], [2, 0], [2, 1], [2, 2], [3, 1], [3, 2], [6, 0], [6, 1], [6, 2], [5, 0]]
[[0, 1], [0, 1], [2, 0], [2, 1], [2, 2], [3, 1], [3, 2], [6, 0], [6, 1], [6, 2], [5, 0]]


### 2-4- Assemblage 

Vous avez définit un certain nombre de fonction qui vous seront utile pour coder la simulation entière. Il vous faut désormais tout assembler pour faire la simulation.
Pour rappel, l'algorithme est le suivant :

À vous de jouer ! Coder tout l'algorithme SAUF L'AFFICHAGE. 

In [1]:
# Simulation

# Initialisation des jeux de données

h = 20
L = 50
proportion = 0.5
T = 150

source = [[h-1, i] for i in range(L)]
ensemble_particules = source.copy()
stockage_ensemble_particules = [[-1]] * T

#assert(len(source)*(1 + proportion)**(T-1) < h*L)
stockage_ensemble_particules[0] = ensemble_particules.copy()

for t in range(1, T):
    
    # Choix des particules à déplacer pour ce tour
    particules_a_deplacer = choisis_particules(ensemble_particules, proportion)
    for particule in particules_a_deplacer :
            
        # Choix de la destination
        destination = trouve_destination(particule, ensemble_particules, h, L)

        # Déplacer ou ajouter la particule
        deplace_particule(particule, destination)
    # Stockage des résultats pour le pas de temps en cours 
    stockage_ensemble_particules[t] = ensemble_particules.copy()

NameError: name 'choisis_particules' is not defined

Regardez si tout est normal sur la dernière image produite.

In [293]:
print(stockage_ensemble_particules[-1])

[[19, 0], [19, 1], [19, 2], [19, 3], [19, 4], [19, 5], [19, 6], [19, 7], [19, 8], [19, 9], [19, 10], [19, 11], [19, 12], [19, 13], [19, 14], [19, 15], [19, 16], [19, 17], [19, 18], [19, 19], [19, 20], [19, 21], [19, 22], [19, 23], [19, 24], [19, 25], [19, 26], [19, 27], [19, 28], [19, 29], [19, 30], [19, 31], [19, 32], [19, 33], [19, 34], [19, 35], [19, 36], [19, 37], [19, 38], [19, 39], [19, 40], [19, 41], [19, 42], [19, 43], [19, 44], [19, 45], [19, 46], [19, 47], [19, 48], [19, 49], [9, 14], [11, 11], [12, 13], [3, 47], [3, 9], [7, 36], [4, 46], [11, 14], [8, 29], [2, 3], [10, 16], [5, 47], [9, 28], [2, 48], [3, 20], [9, 37], [8, 4], [5, 7], [12, 35], [12, 4], [16, 14], [10, 1], [7, 35], [5, 28], [13, 16], [4, 26], [5, 1], [12, 16], [12, 26], [8, 32], [18, 28], [8, 47], [4, 11], [11, 39], [9, 6], [15, 26], [12, 27], [2, 28], [11, 36], [6, 3], [13, 14], [9, 8], [5, 29], [7, 37], [8, 7], [4, 6], [7, 12], [6, 40], [9, 1], [10, 6], [11, 48], [8, 40], [7, 29], [11, 2], [10, 17], [4, 24],

## 3- Affichage

Nous utiliserons ici une méthode d'affichage légèrement différente de celles utilisées précédemment. Nous ne rentrerons pas dans le détail de l'affichage mais il nous faut en premier lieux convertir les données en une matrice ayant les dimensions du milieu et contenant 1 dans les cases où il y a une particule et 0 ailleurs. Vous n'aurez pas à vous pencher sur la suite de l'affichage.

Remplissez la fonction converti_binaire.

In [211]:
def converti_binaire(stockage_ensemble_particules, h, L):
    """
    Prérequis :
    stockage_ensemble_particules est un tableau de particules (i.e. de tableaux
    d'entiers à 2 éléments).
    h et L sont des entiers strictement positifs définissant respectivement la hauteur 
    et la largeur du milieu.
    
    Renvoie un tableau de matrices contenant en chaque élément de ligne i et de colonne j
    0 s'il n'y a pas de particule en position (i,j) et 1 s'il y en a une.
    """
    # Remplir ici
    T = len(stockage_ensemble_particules)
    
    # Création du tableau des images en binaire
    stockage_binaire = [[-1]] * T
    
    for t in range(T) :
        
        # Création de la matrice contenant l'image binaire de l'instant t
        # On initialise toutes les valeurs à 0 qui représente le fond
        image_binaire = [[0]]*h
        
        for l in range(h) :
            image_binaire[l] = [0]*L
        # Pour chaque particule dans la liste des particules au temps t
        for particule in stockage_ensemble_particules[t] :

            # On relève ses coordonnées
            [l_particule, h_particule] = particule

            # On inscrit 1 à la position de la particule dans la matrice
            # de l'image. Ce 1 représente la particule.
            image_binaire[l_particule][h_particule] = 1
        
        # On COPIE l'image binaire dans le tableau des images binaires
        stockage_binaire[t] = image_binaire.copy()
        
    # On renvoie le tableau obtenu
    return stockage_binaire

Testez la fonction converti_binaire avec un cas simple

In [291]:
stockage_binaire = converti_binaire(stockage_ensemble_particules, h, L)

Vous pouvez désormais tester votre simulation ! Cela peut cependant prendre un peu de temps.

In [292]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import animation, rc
import matplotlib.colors as colors

# Création du graphique
fig, ax = plt.subplots()
# Création de la palette de couleur
cmap = colors.LinearSegmentedColormap.from_list("",["green","orange"])

# Affichage 

# initialisation de la commande 
im = ax.imshow(stockage_binaire[0], cmap=cmap, vmin=0, vmax=1)


# fonction appellée pour l'initialisation des données à afficher
def init():
    im.set_data(stockage_binaire[0])
    return [im]

# fonction appellée pour l'actualisation des données à afficher
def animate(i):
    data_slice = stockage_binaire[i]
    im.set_data(data_slice)
    return [im]

# animation 
anim=animation.FuncAnimation(fig, animate, init_func=init,
                             frames=T, interval=30, blit=True,
                             repeat=False)
plt.close()

anim